In [2]:
import gzip
from collections import defaultdict
from sklearn import linear_model
from sklearn.linear_model import LogisticRegression
import csv
import random
import string

In [3]:
def readGz(path):
    for l in gzip.open(path, 'rt', encoding='utf8'):
        yield eval(l)

In [31]:
dataset0 = list(readGz("australian_users_items.json.gz"))
dataset0[0]

{'user_id': '76561197970982479',
 'items_count': 277,
 'steam_id': '76561197970982479',
 'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
 'items': [{'item_id': '10',
   'item_name': 'Counter-Strike',
   'playtime_forever': 6,
   'playtime_2weeks': 0},
  {'item_id': '20',
   'item_name': 'Team Fortress Classic',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '30',
   'item_name': 'Day of Defeat',
   'playtime_forever': 7,
   'playtime_2weeks': 0},
  {'item_id': '40',
   'item_name': 'Deathmatch Classic',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '50',
   'item_name': 'Half-Life: Opposing Force',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '60',
   'item_name': 'Ricochet',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '70',
   'item_name': 'Half-Life',
   'playtime_forever': 0,
   'playtime_2weeks': 0},
  {'item_id': '130',
   'item_name': 'Half-Life: Blue Shift',
   'playtime_forever

In [32]:
len(dataset0)

88310

In [6]:
dataset1 = list(readGz("australian_user_reviews.json.gz"))
dataset1[0]

{'user_id': '76561197970982479',
 'user_url': 'http://steamcommunity.com/profiles/76561197970982479',
 'reviews': [{'funny': '',
   'posted': 'Posted November 5, 2011.',
   'last_edited': '',
   'item_id': '1250',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': 'Simple yet with great replayability. In my opinion does "zombie" hordes and team work better than left 4 dead plus has a global leveling system. Alot of down to earth "zombie" splattering fun for the whole family. Amazed this sort of FPS is so rare.'},
  {'funny': '',
   'posted': 'Posted July 15, 2011.',
   'last_edited': '',
   'item_id': '22200',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': "It's unique and worth a playthrough."},
  {'funny': '',
   'posted': 'Posted April 21, 2011.',
   'last_edited': '',
   'item_id': '43110',
   'helpful': 'No ratings yet',
   'recommend': True,
   'review': 'Great atmosphere. The gunplay can be a bit chunky at times but at the end of the day this 

In [8]:
len(dataset1)

25799

In [9]:
dataset2 = list(readGz("steam_games.json.gz"))
dataset2[0]

{'publisher': 'Kotoshiro',
 'genres': ['Action', 'Casual', 'Indie', 'Simulation', 'Strategy'],
 'app_name': 'Lost Summoner Kitty',
 'title': 'Lost Summoner Kitty',
 'url': 'http://store.steampowered.com/app/761140/Lost_Summoner_Kitty/',
 'release_date': '2018-01-04',
 'tags': ['Strategy', 'Action', 'Indie', 'Casual', 'Simulation'],
 'discount_price': 4.49,
 'reviews_url': 'http://steamcommunity.com/app/761140/reviews/?browsefilter=mostrecent&p=1',
 'specs': ['Single-player'],
 'price': 4.99,
 'early_access': False,
 'id': '761140',
 'developer': 'Kotoshiro'}

In [10]:
len(dataset2)

32135

In [16]:
train = dataset0[0:len(dataset0)//2]
test = dataset0[len(dataset0)//2:]

In [19]:
gamesByUser = defaultdict(set)
usersByGame = defaultdict(set)
for d in train:
    gameList = d['items']
    for game in gameList:
        gamesByUser[d['user_id']].add(game['item_id'])
        usersByGame[game['item_id']].add(d['user_id'])

In [33]:
all_games = usersByGame.keys()
all_users = gamesByUser.keys()
len(all_games)

10937

In [18]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0: return 0
    return numer / denom

In [30]:
id_to_name = {}
name_to_id = {}
for d in dataset0:
    gameList = d['items']
    for game in gameList:
        if game['item_id'] not in id_to_name.keys():
            id_to_name[game['item_id']] = game['item_name']
        if game['item_name'] not in name_to_id.keys():
            name_to_id[game['item_name']] = game['item_id']
    

In [27]:
def mostSimilar(game_id, N):
    similarities = []
    user = usersByGame[game_id]
    for i2 in all_games:
        if i2 == all_games: continue
        sim = Jaccard(user, usersByGame[i2])
        similarities.append((sim,i2,id_to_name[i2]))
    similarities.sort(reverse=True)
    return similarities[:N]

In [51]:
name_to_id['Simply Chess']

'312280'

In [52]:
mostSimilar('312280', 20)

[(1.0, '312280', 'Simply Chess'),
 (0.10564399421128799, '362400', 'Regimental Chess'),
 (0.09873417721518987, '369400', 'Let the Cat In'),
 (0.09831460674157304, '356520', 'Port of Call'),
 (0.09812286689419795, '360870', 'Ultimate Tic-Tac-Toe'),
 (0.09504132231404959, '334030', 'Epic Arena'),
 (0.09351432880844646,
  '378300',
  'The Knobbly Crook: Chapter I - The Horse You Sailed In On'),
 (0.0930576070901034, '380140', 'The Secret of Tremendous Corporation'),
 (0.09281437125748503, '395520', 'Missing Translation'),
 (0.09175257731958762, '314000', 'Pool Nation FX'),
 (0.09049773755656108, '354200', 'Aberoth'),
 (0.08968609865470852, '377430', 'Who Is Mike'),
 (0.08953488372093023, '404480', 'SC2VN - The eSports Visual Novel'),
 (0.08934707903780069, '357480', 'Frozen Free Fall: Snowball Fight'),
 (0.08920863309352518, '399520', 'Comedy Quest'),
 (0.08920704845814978, '321480', 'Sigils of Elohim'),
 (0.0888030888030888, '352160', 'Curse of Mermos'),
 (0.08854961832061069, '375220', 

In [ ]:
def mostSimilar8(recipe, N):
    similarities = []
    ingred = ingsPerItem[recipe]
    for i2 in ingsPerItem:
        if i2 == recipe: continue
        sim = Jaccard(ingred, ingsPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(reverse=True)
    return similarities[:N]